In [125]:
!head -10 ml-100k/u.data


196	242	3	881250949
186	302	3	891717742
22	377	1	878887116
244	51	2	880606923
166	346	1	886397596
298	474	4	884182806
115	265	2	881171488
253	465	5	891628467
305	451	3	886324817
6	86	3	883603013


In [127]:
import pandas as pd
#ratings = pd.read_csv('ml-1m/ratings.dat', delimiter="::", engine="python", header=None)
ratings = pd.read_csv('ml-100k/u.data', delimiter="\t", engine="python", header=None)

In [128]:
ratings.columns = ["UserID::MovieID::Rating::Timestamp".split("::")]
#ratings = ratings.iloc[:1000, :]
ratings.head()


,UserID,MovieID,Rating,Timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [129]:
ratings.isnull().sum()

UserID       0
MovieID      0
Rating       0
Timestamp    0
dtype: int64

In [130]:
import numpy as np
np.unique(ratings['Rating'])

array([1, 2, 3, 4, 5])

* Do we have any duplicate ratings, i.e., a (userid, movieid) combination that is repeated more than once?

In [131]:
user_movie_pairs = zip(ratings['UserID'].values, ratings['MovieID'].values)

In [132]:
print len(user_movie_pairs), len(set(user_movie_pairs))

100000 100000


In [138]:
print ratings['UserID'].describe()
n_user = len(ratings['UserID'].unique())
print "number of users: %s\n" % n_user
print set(ratings['UserID'].unique()) == set(range(1, n_user + 1))
# User ids are range(1, 6041), i.e., are all values between 1 and 6040
# We can use a simple mapping: user_index=UserID-1


count    100000.00000
mean        462.48475
std         266.61442
min           1.00000
25%         254.00000
50%         447.00000
75%         682.00000
max         943.00000
Name: UserID, dtype: float64
number of users: 943

True


In [139]:
#!head ml-1m/movies.dat
!head ml-100k/u.item

1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0
2|GoldenEye (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?GoldenEye%20(1995)|0|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0
3|Four Rooms (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995)|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0
4|Get Shorty (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Get%20Shorty%20(1995)|0|1|0|0|0|1|0|0|1|0|0|0|0|0|0|0|0|0|0
5|Copycat (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Copycat%20(1995)|0|0|0|0|0|0|1|0|1|0|0|0|0|0|0|0|1|0|0
6|Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)|01-Jan-1995||http://us.imdb.com/Title?Yao+a+yao+yao+dao+waipo+qiao+(1995)|0|0|0|0|0|0|0|0|1|0|0|0|0|0|0|0|0|0|0
7|Twelve Monkeys (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Twelve%20Monkeys%20(1995)|0|0|0|0|0|0|0|0|1|0|0|0|0|0|0|1|0|0|0
8|Babe (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Babe%20(1995)|0|0|0|0|1

In [245]:
#movies = pd.read_csv('ml-1m/movies.dat', delimiter='::', engine='python', header=None)
movies = pd.read_csv('ml-100k/u.item', delimiter='|', engine='python', header=None).iloc[:, :5]
#movies.columns = 'MovieID::Title::Genres'.split('::')
movies.columns = 'MovieID::Title::Date::N::Url'.split('::')
movies.head()

print "The following movie id's are missing from movie table"
print sorted(set(range(1, movies.MovieID.max())) - set(movies.MovieID))
print "\nnumber of unique movies: %s\n" %  len(set(movies.MovieID))

mi = ratings['MovieID'].unique()
mi.sort()
print sorted(set(movies.MovieID) - set(mi))
print len(mi)
# movie id have some missing values in addition to the missing values above
# , i.e., there are movies that are not rated by any user.

#movie_index_to_ID = dict(zip(range(movies.MovieID.unique().shape[0]), movies.MovieID))
movie_index_to_ID = dict(zip(range(len(mi)), mi))
#movie_ID_to_index = dict(zip(movies.MovieID, range(movies.MovieID.unique().shape[0])))
movie_ID_to_index = {k: v for v, k in movie_index_to_ID.iteritems()}


The following movie id's are missing from movie table
[]

number of unique movies: 1682

[]
1682


In [145]:
len(movies.MovieID) == len(movies.MovieID.unique())

True

In [146]:
from scipy.sparse import csr_matrix
row = ratings['MovieID'].values
col = ratings['UserID'].values
data = ratings['Rating'].values
ratings_matrix = csr_matrix((data, (row, col)))



In [147]:
ratings_matrix.shape

(1683, 944)

In [44]:
def trace_dot(X, Y):
    """Trace of np.dot(X, Y.T)."""
    return np.dot(X.ravel(), Y.ravel())
norm_X = np.dot(X.data, X.data)
norm_WH = trace_dot(np.dot(np.dot(W.T, W), H), H)
cross_prod = trace_dot((X * H.T), W)
error = np.sqrt(norm_X + norm_WH - 2. * cross_prod)

NameError: name 'X' is not defined

In [45]:
error/X.shape[0]/X.shape[1]

NameError: name 'error' is not defined

In [46]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("WARN")


In [47]:
from pyspark.sql.functions import rand

def kfold_cv_generator(dataset, n_folds=3, seed = None):
    if not seed:
        seed = 0
    h = 1.0 / n_folds
    randCol = "cv_rand"
    df = dataset.select("*", rand(seed).alias(randCol)).cache()
    for i in range(n_folds):
        validateLB = i * h
        validateUB = (i + 1) * h
        condition = (df[randCol] >= validateLB) & (df[randCol] < validateUB)
        validation = df.filter(condition)
        train = df.filter(~condition)
        yield train.drop(randCol), validation.drop(randCol)

In [48]:
#ratings_sp = spark.createDataFrame(ratings)

In [148]:
ratings.columns

Index([u'UserID', u'MovieID', u'Rating', u'Timestamp'], dtype='object')

In [150]:
import os
#TODO: Should use broadcast for movie_ID_to_index?
def parse_data(line):
    fields = line.split("\t")
    user_id = int(fields[0])
    movie_id = int(fields[1])
    rating = float(fields[2])
    ts = fields[3]
    return movie_ID_to_index_bc.value[movie_id], user_id - 1, rating, ts
    
movie_ID_to_index_bc = sc.broadcast(movie_ID_to_index)
#u_data_file = os.path.join("ml-1m", "ratings.dat")
u_data_file = os.path.join("ml-100k", "u.data")
ratings_rdd = sc.textFile(u_data_file).map(parse_data)#.cache().filter(lambda x: x is not None)

In [151]:
x = ratings_rdd.collect()
pd.DataFrame(x).describe()

,0,1,2
count,100000.000000,100000.00000,100000.000000
mean,424.530130,461.48475,3.529860
std,330.798356,266.61442,1.125674
min,0.000000,0.00000,1.000000
25%,174.000000,253.00000,3.000000
50%,321.000000,446.00000,4.000000
75%,630.000000,681.00000,4.000000
max,1681.000000,942.00000,5.000000


number of unique movies: 3883
number of users: 6040

In [153]:
ratings_columns = "UserID::MovieID::Rating::Timestamp".split("::")
ratings_sp = spark.createDataFrame(ratings_rdd, schema=ratings_columns)
ratings_sp.columns

['UserID', 'MovieID', 'Rating', 'Timestamp']

In [55]:
import numpy as np
import os
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating

#u_data_file = os.path.join("ml-1m", "ratings.dat")
#ratings = sc.textFile(u_data_file).map(parse_data).cache()#.filter(lambda x: x is not None)

#ratings_train = ratings.filter(lambda x: x[3] < 8).map(lambda x: x[:-1])
#ratings_test = ratings.filter(lambda x: x[3] >= 8).map(lambda x: x[:-1])

#print("number of train points: %d" % ratings_train.count())
#print("number of test points: %d" % ratings_test.count())

n_folds = 10
MSE = np.zeros(n_folds)
for i, (ratings_train, ratings_test) in enumerate(kfold_cv_generator(ratings_sp, n_folds)):
    
    print ratings_train.count()
    # TODO: Remove timestamp column from the begining
    model = ALS.train(ratings_train.select(['UserID', 'MovieID', 'Rating']), rank = 10, iterations = 10)

    test_data = ratings_test.rdd.map(lambda r: (r[0], r[1]))
    predictions = model.predictAll(test_data).map(lambda r: ((r[0], r[1]), r[2]))
    p = predictions.collect()
    ratesAndPreds = ratings_test.rdd.map(lambda r: ((r[0], r[1]), r[2])).join(predictions)
    MSE[i] = (ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean())**.5

    print("Mean Squared Error = " + str(MSE))


KeyboardInterrupt: 

In [154]:
model = ALS.train(ratings_sp.select(['UserID', 'MovieID', 'Rating']), rank = 10, iterations = 10)

In [157]:
pf = model.productFeatures()
pf_coordinate = pf.map(lambda r: (r[0],  zip(range(len(r[1])), r[1]))).map(lambda r: r[0])

In [158]:
uf = model.userFeatures().collect()

In [59]:
from matplotlib import pyplot as plt

In [60]:
movie_index_list = [i[1] for i in model.recommendProducts(0, 10)]
[movie_index for movie_index_to_ID[movie_index]

SyntaxError: invalid syntax (<ipython-input-60-79f290f9d2d1>, line 2)

### Row-style to coordinate conversion:

In [159]:
"""
m = [
(0, (v00, v01, v02))
(2, (v20, v21, v22))
(5, (v50, v51, v52))
]

=>

[
(0, [(0, v00), (1, v01), (2, v02)])
(2, [(0, v20), (1, v21), (2, v22)])
(5, [(0, v50), (1, v51), (2, v52)])
]

=>

[
(0, 0, v00), (0, 1, v01), (0, 2, v02), 
(2, 0, v20), (2, 1, v21), (2, 2, v22), 

]
"""
def row_style_to_coordinate(m):
    x = m.map(lambda r: (r[0], zip(range(len(r[1])), r[1])))
    return x.flatMap(lambda r: [(r[0], i[0], i[1]) for i in r[1]])

In [160]:
def coordinate_to_sparse(m):
    row, col, data = np.array(m).T
    return csr_matrix((data, (row, col)))


In [161]:
pf = model.productFeatures()
uf = model.userFeatures()
user_features = row_style_to_coordinate(pf)
product_features = row_style_to_coordinate(uf)
print coordinate_to_sparse(user_features.collect()).todense().shape
print coordinate_to_sparse(product_features.collect()).todense().shape

(943, 10)
(1682, 10)


In [172]:
pf_sparse = coordinate_to_sparse(product_features.collect())

In [176]:
pp_sim = np.dot(pf_sparse.todense(), pf_sparse.todense().T)

In [195]:
recom_movie_index = np.array(np.argsort(pp_sim[0, :]))[0][::-1][:5]

In [199]:
recom_movie_index

array([1642, 1462, 1119, 1153, 1285])

In [206]:
movies[0:9]

,MovieID,Title,Date
0,1,Toy Story (1995),01-Jan-1995
1,2,GoldenEye (1995),01-Jan-1995
2,3,Four Rooms (1995),01-Jan-1995
3,4,Get Shorty (1995),01-Jan-1995
4,5,Copycat (1995),01-Jan-1995
5,6,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,01-Jan-1995
6,7,Twelve Monkeys (1995),01-Jan-1995
7,8,Babe (1995),01-Jan-1995
8,9,Dead Man Walking (1995),01-Jan-1995


In [215]:
[movies[movies.MovieID == movie_index_to_ID[i]].Title.tolist()[0] for i in recom_movie_index]

['Angel Baby (1995)',
 'Boys, Les (1997)',
 "I'm Not Rappaport (1996)",
 'Alphaville (1965)',
 'Shall We Dance? (1937)']

In [231]:
recom_movie_index

array([1642, 1462, 1119, 1153, 1285])

In [235]:
help(pd.DataFrame({'MovieID':[movie_index_to_ID[i] for i in recom_movie_index]}).join)

Help on method join in module pandas.core.frame:

join(self, other, on=None, how='left', lsuffix='', rsuffix='', sort=False) method of pandas.core.frame.DataFrame instance
    Join columns with other DataFrame either on index or on a key
    column. Efficiently Join multiple DataFrame objects by index at once by
    passing a list.
    
    Parameters
    ----------
    other : DataFrame, Series with name field set, or list of DataFrame
        Index should be similar to one of the columns in this one. If a
        Series is passed, its name attribute must be set, and that will be
        used as the column name in the resulting joined DataFrame
    on : column name, tuple/list of column names, or array-like
        Column(s) in the caller to join on the index in other,
        otherwise joins index-on-index. If multiples
        columns given, the passed DataFrame must have a MultiIndex. Can
        pass an array as the join key if not already contained in the
        calling DataFram

In [ ]:
(pd.DataFrame({'MovieID':[movie_index_to_ID[i] for i in recom_movie_index]})).join

In [233]:
pd.DataFrame({'MovieID':[movie_index_to_ID[i] for i in recom_movie_index]}).join(movies, how='inner', on='MovieID', lsuffix = '_l')

,MovieID_l,MovieID,Title,Date
0,1643,1644,Sudden Manhattan (1996),13-Jun-1997
1,1463,1464,"Stars Fell on Henrietta, The (1995)",01-Jan-1995
2,1120,1121,"Umbrellas of Cherbourg, The (Parapluies de Che...",05-Apr-1996
3,1154,1155,"Rendezvous in Paris (Rendez-vous de Paris, Les...",28-Jun-1996
4,1286,1287,Ed (1996),08-Mar-1996


In [ ]:
pd.DataFrame({'MovieID':[movie_index_to_ID[i] for i in recom_movie_index]}).join

In [236]:
[movies['Title'].loc[movie_index_to_ID[i]] for i in recom_movie_index]

['Sudden Manhattan (1996)',
 'Stars Fell on Henrietta, The (1995)',
 'Umbrellas of Cherbourg, The (Parapluies de Cherbourg, Les) (1964)',
 'Rendezvous in Paris (Rendez-vous de Paris, Les) (1995)',
 'Ed (1996)']

In [246]:
movies.loc[recom_movie_index]

,MovieID,Title,Date,N,Url
1642,1643,Angel Baby (1995),10-Jan-1997,NaN,http://us.imdb.com/Title?Angel+Baby+(1995/I)
1462,1463,"Boys, Les (1997)",01-Jan-1997,NaN,http://us.imdb.com/M/title-exact?imdb-title-11...
1119,1120,I'm Not Rappaport (1996),13-Nov-1996,NaN,http://us.imdb.com/M/title-exact?I'm%20Not%20R...
1153,1154,Alphaville (1965),01-Jan-1965,NaN,http://us.imdb.com/M/title-exact?Alphaville%20...
1285,1286,Shall We Dance? (1937),01-Jan-1937,NaN,http://us.imdb.com/M/title-exact?Shall%20We%20...


In [256]:
movie_list = movies.loc[recom_movie_index]['Url'].tolist()

In [257]:
def get_movie_image(url):
    return Image(url=get_poster(url, base_url, api_key))

In [273]:
get_poster(movie_list[1], base_url, api_key)

''

In [268]:
s = ''
for url in movie_list:
    s += '<img src="%s" >' % get_poster(url, base_url, api_key)

In [270]:
s

u'<img src="http://image.tmdb.org/t/p/w185/aCvCn3PIdnEAPpHreswg2YXhrEI.jpg" ><img src="" ><img src="http://image.tmdb.org/t/p/w185/xgCiW1oDaJM2zZWrHLlqO3Q8ABm.jpg" ><img src="http://image.tmdb.org/t/p/w185/AlaBz8Krj93TxG3nSyPWVG4tHMV.jpg" ><img src="" >'

<div>
<img src="http://image.tmdb.org/t/p/w185/aCvCn3PIdnEAPpHreswg2YXhrEI.jpg" ><img src="" ><img src="http://image.tmdb.org/t/p/w185/xgCiW1oDaJM2zZWrHLlqO3Q8ABm.jpg" ><img src="http://image.tmdb.org/t/p/w185/AlaBz8Krj93TxG3nSyPWVG4tHMV.jpg" ><img src="" >
</div>

### Get Movie Posters

In [239]:

import requests
import json

response = requests.get('http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)')
print response.url.split('/')[-2]

tt0114709


In [243]:

# Build function to query themoviedb.org's API

# Get base url filepath structure. w185 corresponds to size of movie poster.
api_key = '2554c0fd4b04956e926253a3337c3252'
headers = {'Accept': 'application/json'}
payload = {'api_key': api_key} 
response = requests.get("http://api.themoviedb.org/3/configuration",\
                        params=payload,\
                        headers=headers)
response = json.loads(response.text)
base_url = response['images']['base_url'] + 'w185'

def get_poster(imdb_url, base_url, api_key):
    # Get IMDB movie ID
    response = requests.get(imdb_url)
    movie_id = response.url.split('/')[-2]
    
    # Query themoviedb.org API for movie poster path.
    movie_url = 'http://api.themoviedb.org/3/movie/{:}/images'.format(movie_id)
    headers = {'Accept': 'application/json'}
    payload = {'api_key': api_key} 
    response = requests.get(movie_url, params=payload, headers=headers)
    try:
        file_path = json.loads(response.text)['posters'][0]['file_path']
    except:
        # IMDB movie ID is sometimes no good. Need to get correct one.
        movie_title = imdb_url.split('?')[-1].split('(')[0]
        payload['query'] = movie_title
        response = requests.get('http://api.themoviedb.org/3/search/movie',\
                                params=payload,\
                                headers=headers)
        try:
            movie_id = json.loads(response.text)['results'][0]['id']
            payload.pop('query', None)
            movie_url = 'http://api.themoviedb.org/3/movie/{:}/images'\
                        .format(movie_id)
            response = requests.get(movie_url, params=payload, headers=headers)
            file_path = json.loads(response.text)['posters'][0]['file_path']
        except:
            # Sometimes the url just doesn't work.
            # Return '' so that it does not mess up Image()
            return ''
        
    return base_url + file_path



In [264]:
get_poster(toy_story, base_url, api_key)

u'http://image.tmdb.org/t/p/w185/uMZqKhT4YA6mqo2yczoznv7IDmv.jpg'

In [244]:
from IPython.display import Image
from IPython.display import display

toy_story = 'http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)'
Image(url=get_poster(toy_story, base_url, api_key))

<img src="http://image.tmdb.org/t/p/w185/uMZqKhT4YA6mqo2yczoznv7IDmv.jpg">

In [162]:
product_features_T = product_features.map(lambda r: (r[1], r[0], r[2]))
pp_sim_rdd = matrix_multiply(product_features, product_features_T)
pp_sim = pp_sim_rdd.collect()

KeyboardInterrupt: 

In [1]:
def matrix_multiply(x, y):
    def left_mapper(r):
        i, j, v = r
        return j, (i, v)
    def right_mapper(r):
        j, k, w = r
        return j, (k, w)
    def product_mapper(r):
        _, ((i, v), (k, w)) = r
        return (i, k), v * w
    
    x = x.map(left_mapper)
    y = y.map(right_mapper)
    
    return x.join(y).map(product_mapper).reduceByKey(lambda x, y: x + y).map(lambda r: r[0][0], r[0][1], r[1])

In [21]:
l = [(0, 0, 3.0), (2, 0, -5.0), (3, 2, 1.0),
   (4, 1, 6.0), (6, 2, 2.0), (8, 1, 4.0)]
row = [i[0] for i in l]
col = [i[1] for i in l]
data = [i[2] for i in l]

row, col, data = np.array(l).T
from scipy.sparse import csr_matrix
m1 = csr_matrix((data, (row, col)))


In [22]:
m1.todense()

matrix([[ 3.,  0.,  0.],
        [ 0.,  0.,  0.],
        [-5.,  0.,  0.],
        [ 0.,  0.,  1.],
        [ 0.,  6.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  2.],
        [ 0.,  0.,  0.],
        [ 0.,  4.,  0.]])

In [13]:
m1.T

<3x9 sparse matrix of type '<type 'numpy.float64'>'
	with 6 stored elements in Compressed Sparse Column format>

In [20]:
import numpy as np
np.dot(m1.todense(), m1.todense().T)

matrix([[  9.,   0., -15.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [-15.,   0.,  25.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   1.,   0.,   0.,   2.,   0.,   0.],
        [  0.,   0.,   0.,   0.,  36.,   0.,   0.,   0.,  24.],
        [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   2.,   0.,   0.,   4.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,  24.,   0.,   0.,   0.,  16.]])

In [4]:
mat = sc.parallelize([
   (0, 0, 3.0), (2, 0, -5.0), (3, 2, 1.0),
   (4, 1, 6.0), (6, 2, 2.0), (8, 1, 4.0)]
)

In [17]:
mat_t = sc.parallelize([(j, i, v) for i, j, v in l])

In [26]:
matrix_multiply(mat, mat_t).collect()

[((2, 0), -15.0),
 ((6, 3), 2.0),
 ((0, 0), 9.0),
 ((4, 4), 36.0),
 ((8, 8), 16.0),
 ((0, 2), -15.0),
 ((3, 3), 1.0),
 ((4, 8), 24.0),
 ((8, 4), 24.0),
 ((6, 6), 4.0),
 ((2, 2), 25.0),
 ((3, 6), 2.0)]

In [ ]:
number of unique movies: 3883
number of users: 6040

In [68]:
set(x) == set(range(6040))

True

In [69]:
set(y) == set(range(3706))

True

In [66]:
x = [i[0] for i in pf]

In [67]:
y = [i[0] for i in uf]

In [ ]:
pd.DataFrame(x).describe()

In [ ]:
max(sorted(y))

In [ ]:
pd.DataFrame(y).describe()

In [ ]:
plt.plot(sorted(x), '.')
plt.show()

In [ ]:
dir(model)

In [ ]:
np.mean([i[1] for i in p])

In [ ]:
from pyspark.ml.tuning import CrossValidator
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder



lr = LogisticRegression(maxIter=10)

paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, np.logspace(-2, 2, 5)) \
    .build()

cnt = 0

class Evaluator:
    def evaluate(self, dataset):
        global cnt
        cnt += 1
        print dataset.count()
        print cnt
        #print dataset['features'].collect()
        #print dataset['label']
        print
        return 1
    
crossval = CrossValidator(estimator=lr,
                          estimatorParamMaps=paramGrid,
                          #evaluator=BinaryClassificationEvaluator(),
                          evaluator=Evaluator(),
                          numFolds=3)  # use 3+ folds in practice

# Run cross-validation, and choose the best set of parameters.
cvModel = crossval.fit(training)



In [ ]:
import os
spark_home = os.environ['SPARK_HOME']
training = spark.read.format("libsvm").load(os.path.join(spark_home, "data/mllib/sample_libsvm_data.txt"))


In [ ]:
training